---

In [148]:
#importing all necessary packages
import numpy as np
import sys
from copy import deepcopy
import heapq
import json
import math
from collections import deque
import itertools

### PuzzleNode Class

`PuzzleNode` is a representation of a node in the A* search algorithm tailored for the \(n^2-1\) puzzle.

### Attributes:

- **`state`**: 
  - Reflects the current configuration of the puzzle board.
  
- **`parent`**: 
  - Points to the preceding node which led to the current state.
  
- **`path_cost`**: 
  - Tracks the cumulative cost to reach this node from the start.
  
- **`heuristic_cost`**: 
  - Estimates the cost from this node to the goal using a heuristic.
  
- **`total_cost`**: 
  - A summation of `path_cost` and `heuristic_cost`, giving an approximation of the most economical route passing through this node. It's pivotal for determining the next node to expand in a priority queue.
  
- **`pruned`**: 
  - A boolean that indicates if the node should be skipped in the search.

### Key Methods:

- **`__init__`**: 
  - Constructs a `PuzzleNode` with the ability to set its initial attributes.
  
- **`__lt__`**: 
  - Enables comparison between two nodes based on their `total_cost`, aiding in their prioritization.
  
- **`__str__`**: 
  - Renders the board's current state in a human-friendly manner, transforming each row into a formatted string.

In essence, `PuzzleNode` serves as a building block in the A* algorithm, holding the puzzle's state, its relationship to other states, and providing utility methods for operations like comparison and display.



In [149]:
class PuzzleNode:
    """
    PuzzleNode represents a node in the A* search algorithm for solving the n^2-1 puzzle.
    """

    def __init__(self, state=[], parent=None, path_cost=0, heuristic_cost=0):
        """
        Initialize a PuzzleNode.

        Args:
            state (list of lists): The current state of the puzzle board. Default is an empty board.
            parent (PuzzleNode): The parent node that led to this node. Default is None.
            path_cost (int): The cost of reaching this node from the initial node. Default is 0.
            heuristic_cost (int): The estimated cost from this node to the goal node using a heuristic function. Default is 0.
        """
        self.state = state
        self.parent = parent
        self.path_cost = path_cost
        self.heuristic_cost = heuristic_cost
        #This value represents the approximate cost of the most economical solution passing through this node. 
        #It will be employed for comparisons to identify the node in the priority queue that needs to be expanded
        self.total_cost = path_cost + heuristic_cost  
        #This flag is used to signal whether or not the node can be disregarded in the search
        self.pruned = False  

    def __lt__(self, other):
        """
        Compare two nodes based on their total cost (f_cost).

        Args:
            other (PuzzleNode): Another node to compare to.

        Returns:
            bool: True if this node's total cost is less than the other node's total cost, otherwise False.
        """
        return self.total_cost < other.total_cost

    def __str__(self):
        """
        Generate a human-readable representation of the current board state.

        Returns:
            str: A string representing the current board state.
        """
        board_rows = []
        for row in self.state:
            # Concatenate cells in the row with formatting
            row_str = "".join(
                str(cell).replace(",", "").replace("[", "|").replace("]", "|")
                for cell in row
            )
            board_rows.append(row_str)
        
        # Join rows with newlines to create the final board representation
        board_str = "\n".join(board_rows)
        
        return board_str

### Heuristics for n-Puzzle Problem

Heuristics are used in search algorithms to estimate the cost from the current state to the goal. They don't give the exact cost but provide an estimate that can guide the search. The n-Puzzle problem, which involves sliding tiles on a board to achieve a goal configuration, can benefit from various heuristics. Below are three such heuristics explained in detail.

### 1. Misplaced Tiles Heuristic (`h1`)

### Mechanism:
The misplaced tiles heuristic simply counts the number of tiles that are not in their goal position. This heuristic is admissible, meaning it never overestimates the true cost to reach the goal, as moving a misplaced tile to its correct position will require at least one move.

### 2. Manhattan Distance Heuristic (`h2`)

### Mechanism:
The Manhattan distance is the distance between two points in a grid-based path (like Manhattan street grids) and is computed as the sum of the absolute differences of their coordinates. In the context of the n-Puzzle, it represents the minimum number of moves required to get a tile to its goal position, assuming no other tiles are in the way. For each tile, the distance is the sum of the horizontal and vertical distances to its goal position. The heuristic is the sum of these distances for all tiles. Like the misplaced tiles heuristic, this heuristic is also admissible.

### 3. Disjoint Pattern Heuristic (`h3`)

### Mechanism:
The disjoint pattern heuristic divides the tiles into two or more groups, and for each group, it calculates a heuristic value (usually the Manhattan distance) as if the other groups didn't exist. By treating the groups as independent, the heuristic can be more informed than just considering each tile individually. The heuristic value for the state is then the sum of the heuristic values for each group. This heuristic can be more accurate than the previous two but is also more computationally intensive.


In [150]:
# Define the PuzzleNode class and other code as shown earlier
# Misplaced tiles heuristic
def h1(state):
    '''
    This heuristic function calculates the number of misplaced tiles in the current state
    when compared to the goal state. The goal state is assumed to be a square grid with
    integers from 0 to N^2 - 1 (where N is the dimension of the grid).

    Args:
        state (list of lists): The current state as a 2D list.

    Returns:
        int: The number of tiles that are in the wrong position in the current state
             when compared to the goal state. A lower value indicates a closer match to the goal state.
    '''
    
    h = 0
    # Define the goal state as a 2D list
    goal = [[(i * len(state) + j) % (len(state) * len(state)) for j in range(len(state))] for i in range(len(state))]
    
    # Loop through the current state and compare it to the goal state
    for i in range(len(state)):
        for j in range(len(state)):
            # If the tile is not in the correct position (not equal to 0 or the corresponding goal state value), increment h
            if state[i][j] != 0 and state[i][j] != goal[i][j]:
                h += 1
    
    return h

    
# Manhattan distance heuristic
def h2(state):
    """
    This function calculates the Manhattan distance from the solved state, given the board state.

    The Manhattan distance is the sum of the absolute differences between the current position of each tile
    on the board and its goal position in a solved configuration. It measures how far the tiles are from their
    correct positions.

    Input:
        - state: The board state as a flat list or list of lists. It represents the current configuration of the board.

    Output:
        - h: The Manhattan distance from the solved configuration, which is a heuristic estimate of the number
             of moves needed to reach the solved state.
    """

    h = 0  # Initialize the Manhattan distance to 0

    # Create a goal state configuration for comparison
    goal = [[(i * len(state) + j) % (len(state) * len(state)) for j in range(len(state))] for i in range(len(state))]

    # Iterate through the current state to calculate Manhattan distance
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] == 0:
                continue  # Skip the empty tile
            r, c = divmod(state[i][j], len(state) )  # Calculate the goal position of the current tile
            h += abs(i - r) + abs(j - c)  # Add the Manhattan distance for this tile to the total distance

    return h  # Return the total Manhattan distance

            

# Disjoint pattern heuristic
def h3(state):
    '''
    Disjoint pattern heuristic for the n-Puzzle problem.

    This heuristic calculates the sum of misplaced tiles in two disjoint patterns
    of the n-Puzzle, where the puzzle is divided into two parts: 
    1. The lower pattern contains numbers from 1 to (n^2 / 2), and
    2. The upper pattern contains numbers from (n^2 / 2 + 1) to (n^2 - 1).

    Args:
        state (list of lists): The current state of the n-Puzzle represented as a 2D list.

    Returns:
        int: The sum of misplaced tiles in the two disjoint patterns, which represents
        an estimate of the cost to reach the goal state.\
    '''
    
    # Find n and the cutoff between the disjoint patterns
    n = len(state)
    cutoff = round(n**2 / 2)

    # Calculate the two relaxed subproblems
    state_lower = [x for row in state for x in row if 1 <= x <= cutoff]
    state_upper = [x for row in state for x in row if cutoff < x <= n * n - 1]

    # Calculate the number of misplaced tiles in each subproblem
    misplaced_lower = sum(x != i for i, x in enumerate(state_lower, start=1))
    misplaced_upper = sum(x != i for i, x in enumerate(state_upper, start=cutoff + 1))

    # Return the sum of misplaced tiles in both subproblems
    return misplaced_lower + misplaced_upper


    


# List of heuristics
heuristics = [h1, h2, h3]



# Testing the heuristics functionality
test1 = [
    [3, 7, 0],
    [5, 4, 8],
    [6, 2, 1]
]
print("Test1 State:")
for row in test1:
    print(row)
print("Misplaced tiles (h1):", h1(test1))
print("Manhattan Distance (h2):", h2(test1))
print("Disjoint pattern (h3):", h3(test1))


Test1 State:
[3, 7, 0]
[5, 4, 8]
[6, 2, 1]
Misplaced tiles (h1): 6
Manhattan Distance (h2): 12
Disjoint pattern (h3): 8


### Puzzle Solvability Checker (`is_solvable`)

### Purpose:
The `is_solvable` function determines whether a given n-Puzzle configuration is solvable. Not every random configuration of the n-Puzzle is solvable. This function assesses solvability by counting inversions — instances where a tile has a lower value but appears before a tile with a higher value. This inversion count, combined with the board's dimensions and the position of the blank tile, can determine if a solution exists.

### Mechanism:
1. **Inversions**: The function counts how many pairs of tiles are in the "wrong order". If tile A comes before tile B, but A has a larger value than B, this counts as an inversion.
2. **Board Dimensions and Blank Tile**: The position of the blank tile (or zero tile) and the width of the board play a role in determining solvability. The parity (odd/even nature) of inversions, combined with these factors, provides the answer.

### Rules for Solvability:
- For a board with an **odd width**, the puzzle is solvable if there's an **even number of inversions**.
- For a board with an **even width**:
  - If the blank tile is on an **even row** counting from the bottom (last row is considered odd, the one above even, and so on), then the puzzle is solvable if there's an **even number of inversions**.
  - If the blank tile is on an **odd row** counting from the bottom, the puzzle is solvable if there's an **odd number of inversions**.


In [151]:
def is_solvable(state):
    
    """
    Determine the solvability of a puzzle by analyzing the count of inversions (the total number of tiles that have lower values and appear before tiles with higher values). This is assessed in conjunction with the board's dimensions.
    
    Input:
        - state (list of lists): the current configuration of the game board
    
    Output:
        - True or False (boolean): Returns True if the puzzle is solvable, and False if it is not
    """
    
    # Verify if the provided state is in the correct format
    if type(state) != list:
        raise TypeError('The state must be a list of lists')

    # Initialize a counter to keep track of the inversions.
    inversions = 0
    
    # Obtain the width of the board and the total number of tiles
    board_width = len(state)
    num_tiles = board_width**2
    
    # If the puzzle does not match the expected dimensions, reshape it
    state = np.array(state)
    if state.shape != (board_width, board_width):
        state = state.reshape(board_width, board_width) 
    
    # Determine the row where the empty tile is, which is crucial for solvability assessment
    blank_tile_row = np.where(state == 0)[0][0]
    
    # Flatten the state into a list to simplify the computation of inversions
    tiles = [*state.flatten()]
    
    # Calculate the count of inversions
    for i in range(num_tiles - 1):
        for j in range(i + 1, num_tiles):
            if (tiles[j] and tiles[i] != 0) and tiles[i] > tiles[j]:
                inversions += 1
                
    # Conditions for solvability
    
    # For a board with an odd width, solvability requires an even count of inversions
    if (board_width % 2 != 0) and (inversions % 2 == 0):
        return True
    
    # For a board with even width and the empty tile on an even row from the bottom, solvability demands an odd count of inversions
    elif (board_width % 2 == 0) and (blank_tile_row % 2 != 0) and (inversions % 2 != 0):
        return True 
    
    # For a board with even width and the empty tile on an odd row from the bottom, solvability requires an even count of inversions
    elif (board_width % 2 == 0) and (blank_tile_row % 2 == 0) and (inversions % 2 == 0):
        return True
    
    # If none of the above conditions are met, the puzzle is not solvable
    else:
        return False


### Successor State Generator (`successors_maker`) [Helper function]

### Purpose:
The `successors_maker` function computes possible successor states for a given n-Puzzle configuration. It does this by moving the empty spot (denoted by `0`) in all valid directions based on its current position. Essentially, this function models the basic actions a player can take at any given state of the game.

### Mechanism:
1. **Locating the Empty Spot**: The function begins by identifying the position of the empty spot on the board.
2. **Possible Moves**: Depending on the position of the empty spot, it can be moved in up to four directions: up, down, left, and right. However, edge and corner positions limit this movement.
3. **Generating Successors**: For each valid move, the function creates a new board configuration (a successor) by swapping the position of the empty spot with the appropriate neighboring tile.

### Rules for Movement:
- The empty spot can move **upwards** unless it's on the top row.
- It can move **leftwards** unless it's in the leftmost column.
- It can slide **downwards** if it's not on the bottom row.
- It can be pushed **rightwards** unless it's in the rightmost column.


In [152]:
def successors_maker(state):
    
    """
    Compute potential successor states by shifting the empty spot in valid directions based on its current position.
    
    Input: 
        - state (list of lists): the current arrangement of the game board
    
    Output:
        - successors_list: a collection of achievable successor board states after performing permissible actions
    """
    
    # Identify the current coordinates of the empty tile
    blank_position = np.where(np.array(state) == 0)
    row, col = blank_position[0][0], blank_position[1][0]
    
    # Prepare a list to store the potential successor states
    successors_list = []
     
    # Shift the empty tile upwards if it's not already at the top row
    if row != 0: 
        
        # Clone the state to safely execute a valid action
        next_state = deepcopy(state)
        
        # Relocate the empty tile upwards and record the resulting state
        next_state[row][col], next_state[row-1][col] = next_state[row-1][col], next_state[row][col]
        successors_list.append(next_state)
      
    # Move the empty tile leftwards if it's not in the leftmost column
    if col != 0:
        
        # Make a duplicate of the state to apply a valid action
        next_state = deepcopy(state)
        
        # Transfer the empty tile to the left and record the successor state
        next_state[row][col], next_state[row][col-1] = state[row][col-1], state[row][col]
        successors_list.append(next_state)
    
    # Slide the empty tile downwards if it's not already at the bottom row
    if row != len(state) - 1: 
        
        # Create a copy of the state to execute a valid action
        next_state = deepcopy(state)
        
        # Relocate the empty tile downwards and record the successor state
        next_state[row][col], next_state[row+1][col] = state[row+1][col], state[row][col]
        successors_list.append(next_state)

    # Push the empty tile to the right if it's not in the rightmost column
    if col != len(state) - 1: 
        
        # Duplicate the state to safely perform a valid action
        next_state = deepcopy(state) 
        
        # Shift the empty tile to the right and record the successor state
        next_state[row][col], next_state[row][col+1] = state[row][col+1], state[row][col]
        successors_list.append(next_state)
    
    # Provide the list of potential successor states
    return successors_list


### Puzzle Solver (`solvePuzzle`)

### Purpose:
The `solvePuzzle` function employs the A* search algorithm to determine the optimal solution for a given n-Puzzle configuration. It utilizes heuristic functions to guide the search, aiming to find the most efficient path from an initial state to the goal state.

### Mechanism:
1. **Initial Checks**: The function initially ensures that the provided puzzle state is solvable. If it isn't, an error code is returned. Furthermore, the function checks that the provided state is a valid n-Puzzle configuration.
2. **Heuristic Guidance**: The provided heuristic function estimates the h-cost. Different heuristics can result in variations in search performance.
3. **Path Reconstruction**: Once the goal state is reached, the function backtracks from the goal to the initial state to reconstruct the optimal path.
4. **Astar Search**: The function explores potential successor states by moving the empty tile in various directions, always selecting the state with the lowest estimated cost to the goal (f-cost) as the next state to explore. This f-cost is a combination of the path cost from the starting state to the current state (g-cost) and the estimated cost from the current state to the goal (h-cost, obtained from the heuristic).

#### Mechanism of A*:
1. **Initialization**: Start with the initial state and compute its \(f\)-cost using the provided heuristic. This state is added to the frontier (a priority queue).
2. **Loop Until Goal is Found**: The state with the lowest \(f\)-cost is always selected for expansion. If this state is the goal, the algorithm terminates.
3. **State Expansion**: The `successors_maker` function generates successor states by moving the empty tile in all valid directions. For each successor, its \(f\)-cost is computed.
4. **Path Cost and Heuristic Evaluation**: 
   - The g-cost for a successor is the g-cost of the current state plus one (since each move has a cost of one).
   - The h-cost is computed using the provided heuristic function.
   - The sum gives the \(f\)-cost for the successor.
5. **Update Frontier**: If a successor state has not been explored or if a better path to it is found (a lower \(f\)-cost), it is added to the frontier.
6. **Avoid Revisits**: States are marked as explored after expansion to prevent redundant exploration. The explored set also helps in pruning paths that have higher costs to already explored states.
7. **Path Reconstruction**: Once the goal state is reached, the function backtracks from the goal to the initial state to reconstruct the optimal path.



In [153]:
def solvePuzzle(state, heuristic, show=False):
    
    """
    Find the solution for a puzzle with a dimension greater than 2, but note that for dimensions greater than 4, the search may be less optimal.

    A* puzzle-solving algorithm adapted from Class Session 3.1

    Inputs:
        - state: The initial puzzle state, represented as a list of lists.
        - heuristic: A reference to a heuristic function, selected from those defined in Question 2.

    Outputs:
        - steps: The number of steps required to optimally solve the puzzle, excluding the initial state.
        - exp: The count of nodes expanded during the search to reach the solution.
        - max_frontier: The maximum size of the frontier queue throughout the search.
        - opt_path: The optimal path, presented as a list of list of lists. This means that opt_path[:,:,i] will provide a list of lists representing the state of the board at the ith step of the solution.
        - err: An error code. Return -1 if the state doesn't have the appropriate size and dimension. For the extension task, return -2 if the state is unsolvable.
    """
    
    # Initialize variables
    steps = 0
    exp = 0
    max_frontier = 0
    err = 0

    # Extension 1 (function defined above)
    # Handle cases where the puzzle is unsolvable
    if not is_solvable(state):
        print("The puzzle is unsolvable")
        opt_path, err = None, -2
        return steps, exp, max_frontier, opt_path, err

    # Verify the dimensions of the puzzle
    board_size = len(state)

    # Ensure the puzzle board is a square (n*n) board
    for row in state:
        if len(row) != board_size:
            opt_path, err = None, -1
            return steps, exp, max_frontier, opt_path, err

    # Check if the state contains the right numbers
    correct_state = [*range(board_size**2)]
    sorted_state = sorted([*itertools.chain(*state)])

    # Return an error if the sorted state does not match the correct set
    if sorted_state != correct_state:
        opt_path, err = None, -1
        return steps, exp, max_frontier, opt_path, err

    # Define the goal state
    goal_state = [[(i * len(state) + j) % (len(state) * len(state)) for j in range(len(state))] for i in range(len(state))]

    # Define the root node
    initial_node = PuzzleNode(state, heuristic_cost=heuristic(state))

    # Create a dictionary to track visited PuzzleNodes, preventing repeated states, including those that result from undoing the previous move
    explored = {str(initial_node.state): initial_node}

    # Define the frontier as a Priority Queue, and add the initial node to it, increasing the size of the frontier
    frontier = [initial_node]

    # as long as the frontier has nodes
    while frontier:
        # The node with the lowest f_cost
        current_node = heapq.heappop(frontier)

        # Check if the node is marked for pruning
        if current_node.pruned:
            continue 
        
        # Check if we got to the goal state
        if current_node.state == goal_state:
            break 

        # Generate successor states by moving the empty tile in all possible directions
        successor_states = successors_maker(current_node.state)

        # increase the relevant counter since a node was expanded to yield successor states
        exp += 1

        # Evaluate the next moves
        for successor in successor_states:
            # Calculate the tentative path cost to reach the node
            g_cost = current_node.path_cost + 1

            # check whether we need to update the path if the child is already explored
            if str(successor) in explored:
                if explored[str(successor)].path_cost > g_cost:
                    explored[str(successor)].pruned = True
                else:
                    continue  # Ignore this node

            # Create a new successor node
            child_node = PuzzleNode(successor, current_node, g_cost, heuristic(successor))
            # Add the successor node to the frontier and keep the lowest f_cost node on top
            heapq.heappush(frontier, child_node)

            # Update the max frontier
            max_frontier = max(len(frontier), max_frontier)

            # set the child node as explored
            explored[str(successor)] = child_node

    # Display the progression of optimal path from initial state to goal state
    if show:

        # Compile the most optimal path to reach the goal state
        opt_path = [current_node]
        while current_node.parent:
            opt_path.append(current_node.parent)
            current_node = current_node.parent

        # Reverse the order of the paths to show the path from initial state to goal state
        opt_path.reverse()

        tag_str = ''

        for node in opt_path:
            print(node, '\n')

        opt_path = [node.state for node in opt_path]

    # If not dsiplaying
    else:
        # Compile the most optimal path to reach the goal state
        opt_path = [current_node.state]
        while current_node.parent:
            opt_path.append((current_node.parent).state)
            current_node = current_node.parent

        # Reverse the order of the paths to show the path from initial state to goal state
        opt_path.reverse()

    return len(opt_path) - 1, exp, max_frontier, opt_path, err


<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [154]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [155]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [156]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [157]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

In [158]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

The puzzle is unsolvable
